### A Simplied Interface to Text Classification With Hugging Face Transformers in TensorFlow Using [ktrain](https://github.com/amaiya/ktrain)

*ktrain* requires TensorFlow 2.

In [2]:
!pip3 install -q tensorflow_gpu==2.1.0

In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.0


We then need to install *ktrain* library using pip.

In [4]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 118kB/s 
     |████████████████████████████████| 421.8MB 23kB/s 
     |████████████████████████████████| 6.7MB 37.3MB/s 
     |████████████████████████████████| 983kB 36.7MB/s 
     |████████████████████████████████| 245kB 43.2MB/s 
     |████████████████████████████████| 757kB 38.4MB/s 
     |████████████████████████████████| 471kB 39.5MB/s 
     |████████████████████████████████| 890kB 36.1MB/s 
     |████████████████████████████████| 3.0MB 40.1MB/s 
     |████████████████████████████████| 1.1MB 32.0MB/s 


### Load a Dataset Into Arrays

In [5]:
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


## STEP 1:  Preprocess Data and Create a Transformer Model

We will use [DistilBERT](https://arxiv.org/abs/1910.01108).

In [6]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=train_b.target_names)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 308
	95percentile : 837
	99percentile : 1938


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 343
	95percentile : 979
	99percentile : 2562


## STEP 2:  Train the Model

In [7]:
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Train for 377 steps, validate for 47 steps
Epoch 1/4
377/377 [==============================] - 330s 875ms/step - loss: 0.5818 - accuracy: 0.8175 - val_loss: 0.2078 - val_accuracy: 0.9308
Epoch 2/4
377/377 [==============================] - 319s 847ms/step - loss: 0.1415 - accuracy: 0.9575 - val_loss: 0.1481 - val_accuracy: 0.9621
Epoch 3/4
377/377 [==============================] - 319s 847ms/step - loss: 0.0477 - accuracy: 0.9863 - val_loss: 0.1427 - val_accuracy: 0.9581
Epoch 4/4
377/377 [==============================] - 319s 845ms/step - loss: 0.0186 - accuracy: 0.9965 - val_loss: 0.1306 - val_accuracy: 0.9667


## STEP 3: Evaluate and Inspect the Model

In [8]:
learner.validate(class_names=t.get_classes())

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.95      0.94       319
         comp.graphics       0.98      0.97      0.97       389
               sci.med       0.97      0.97      0.97       396
soc.religion.christian       0.98      0.97      0.98       398

              accuracy                           0.97      1502
             macro avg       0.97      0.97      0.97      1502
          weighted avg       0.97      0.97      0.97      1502



array([[303,   0,   8,   8],
       [ 10, 376,   3,   0],
       [  4,   6, 386,   0],
       [  8,   2,   1, 387]])

Let's examine the validation example about which we were the most wrong.

In [9]:
learner.view_top_losses(n=1, preproc=t)

----------
id:756 | loss:7.77 | true:soc.religion.christian | pred:comp.graphics)



In [10]:
print(x_test[371])

From: kempmp@phoenix.oulu.fi (Petri Pihko)
Subject: Re: Consciousness part II - Kev Strikes Back!
Organization: University of Oulu, Finland
X-Newsreader: TIN [version 1.1 PL9]
Lines: 30

Scott D. Sauyet (SSAUYET@eagle.wesleyan.edu) wrote:
> In <1993Apr21.163848.8099@cs.nott.ac.uk> 
> Kevin Anthony (kax@cs.nott.ac.uk) writes:

> > Firstly, I'm not impressed with the ability of algorithms. They're
> > great at solving problems once the method has been worked out, but not
> > at working out the method itself.
>   [ .. crossword example deleted ... ]

> Have you heard of neural networks?  I've read a little about them, and
> they seems to overcome most of your objections.

I'm sure there are many people who work with neural networks and
read this newsgroup. Please tell Kevin what you've achieved, and
what you expect.

> I am not saying that NNs will solve all such problems, but I think
> they show that it is not as hard as you think to come up with
> mechanical models of consciousness.

In

This post talks more about computing than `alt.atheism` (the true category), so our model placed it into the only computing category available to it: `comp.graphics`

## STEP 4: Making Predictions on New Data in Deployment

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [12]:
predictor.predict('Jesus Christ is the central figure of Christianity.')

'soc.religion.christian'

In [13]:
# predicted probability scores for each category
predictor.predict_proba('Jesus Christ is the central figure of Christianity.')

array([3.5261922e-03, 3.2802156e-04, 3.8910788e-04, 9.9575669e-01],
      dtype=float32)

In [14]:
predictor.get_classes()

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

As expected, `soc.religion.christian` is assigned the highest probability.

Let's invoke the `explain` method to see which words contribute most to the classification.

We will need a forked version of the **eli5** library that supportes TensorFlow Keras, so let's install it first.

In [15]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1


In [16]:
predictor.explain('Jesus Christ is the central figure in Christianity.')

The words in the darkest shade of green contribute most to the classification and agree with what you would expect for this example.

We can save and reload our predictor for later deployment.

In [17]:
predictor.save('/tmp/my_distilbert_predictor')

In [18]:
reloaded_predictor = ktrain.load_predictor('/tmp/my_distilbert_predictor')

In [19]:
reloaded_predictor.predict('My computer monitor is really blurry.')

'comp.graphics'